<a href="https://colab.research.google.com/github/wooookim/ML-DL/blob/main/%EC%84%9C%EC%9A%B8%EC%8B%9C_%ED%8F%89%EA%B7%A0_%EA%B8%B0%EC%98%A8_%EC%98%88%EC%B8%A1_14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
data = pd.read_csv('/content/train.csv')
submission = pd.read_csv('/content/sample_submission.csv')

In [3]:
# '일시' 열을 날짜 데이터로 변환
data['일시'] = pd.to_datetime(data['일시'])

# '일시' 컬럼에서 '월', '일' 열 추가
data['월'] = data['일시'].dt.month
data['일'] = data['일시'].dt.day

# 월, 일별 중앙값 계산
monthly_medians = data.groupby('일').transform('median')
daily_medians = data.groupby('일').transform('median')

<ipython-input-3-f0d9f2a84bd0>:9: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  monthly_medians = data.groupby('일').transform('median')
<ipython-input-3-f0d9f2a84bd0>:10: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  daily_medians = data.groupby('일').transform('median')


In [4]:
from datetime import datetime

current_year = datetime.now().year

# 2월 29일에 해당하는 행 선택
selected_rows = data[(data['일시'].dt.month == 2) & (data['일시'].dt.day == 29)]

# 29일에 해당하는 16개 행
# 23011 - 16 = 22995

# 선택한 행의 인덱스를 사용하여 삭제
data = data.drop(selected_rows.index)

print(data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22995 entries, 0 to 23010
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   일시      22995 non-null  datetime64[ns]
 1   최고기온    22992 non-null  float64       
 2   최저기온    22992 non-null  float64       
 3   일교차     22991 non-null  float64       
 4   강수량     9142 non-null   float64       
 5   평균습도    22995 non-null  float64       
 6   평균풍속    22991 non-null  float64       
 7   일조합     22877 non-null  float64       
 8   일사합     18137 non-null  float64       
 9   일조율     22630 non-null  float64       
 10  평균기온    22995 non-null  float64       
 11  월       22995 non-null  int64         
 12  일       22995 non-null  int64         
dtypes: datetime64[ns](1), float64(10), int64(2)
memory usage: 2.5 MB
None


윤년 데이터 빼고 365로 나눠서 모델에 적용시키기

In [5]:
train = data[['최고기온', '최저기온', '일교차', '강수량', '평균습도', '평균풍속', '일조합', '일사합', '일조율']]

target = data['평균기온']

In [6]:
# 결측값 대체
data.fillna(daily_medians, inplace=True)

from xgboost import XGBRegressor
xgb = XGBRegressor()
xgb.fit(train, target)

pred_xgb = xgb.predict(train)

from sklearn.metrics import mean_squared_error
xgb_mae = mean_squared_error(target, pred_xgb)
print(xgb_mae)

submission['평균기온'] = pred_xgb[:358]
print(submission.head(10))

0.14312530580469168
           일시      평균기온
0  2023-01-01 -2.011936
1  2023-01-02 -1.901870
2  2023-01-03  3.772155
3  2023-01-04  6.986705
4  2023-01-05 -4.083561
5  2023-01-06 -5.160858
6  2023-01-07  1.405020
7  2023-01-08  1.828838
8  2023-01-09  2.475737
9  2023-01-10  1.886170


타겟이 정렬된 형태라서 똑같이 나오나...?

스케일링 적용해서 결과 확인해보기

In [8]:
# 결측값 대체
data.fillna(daily_medians, inplace=True)

from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(train, target)

pred_rf = xgb.predict(train)

from sklearn.metrics import mean_squared_error
rf_mae = mean_squared_error(target, pred_rf)
print(rf_mae)

submission['평균기온'] = pred_rf[:358]
print(submission.head(10))

ValueError: ignored

In [11]:
from sklearn.impute import SimpleImputer

# 중앙값으로 NaN 대체
imputer = SimpleImputer(strategy='median')
train_imputed = imputer.fit_transform(train)

# 나머지 모델 학습 등의 과정은 그대로 유지
rf = RandomForestRegressor()
rf.fit(train_imputed, target)

pred_rf = rf.predict(train_imputed)

rf_mae = mean_squared_error(target, pred_rf)
print(rf_mae)

submission['평균기온'] = pred_rf[:358]
print(submission.head(30))

0.03881840369645575
            일시    평균기온
0   2023-01-01  -1.672
1   2023-01-02  -2.004
2   2023-01-03   3.749
3   2023-01-04   6.793
4   2023-01-05  -4.161
5   2023-01-06  -5.258
6   2023-01-07   1.186
7   2023-01-08   1.791
8   2023-01-09   2.243
9   2023-01-10   2.009
10  2023-01-11  -1.961
11  2023-01-12  -3.703
12  2023-01-13  -3.389
13  2023-01-14  -1.746
14  2023-01-15  -5.045
15  2023-01-16  -4.941
16  2023-01-17  -5.453
17  2023-01-18  -2.795
18  2023-01-19   2.095
19  2023-01-20  -0.712
20  2023-01-21  -6.455
21  2023-01-22  -9.380
22  2023-01-23 -13.142
23  2023-01-24 -12.591
24  2023-01-25  -7.332
25  2023-01-26  -7.521
26  2023-01-27 -10.795
27  2023-01-28 -11.248
28  2023-01-29  -7.859
29  2023-01-30  -0.373


**Nan 값 처리**

에러 메시지에서 나타나는 Input X contains NaN는 RandomForestRegressor 모델이 NaN 값을 포함하는 입력 데이터를 처리하지 못했기 때문에 발생한 것이에요.

일반적으로 fillna 메서드는 데이터프레임의 NaN 값을 특정 값으로 채우는 데에 사용되지만, 모델 학습을 위한 입력 데이터에 대해 이 작업을 수행할 때는 주의해야 해요. fillna를 사용한 후에도 NaN 값이 남아있을 수 있으며, 이는 모델이 처리하지 못하는 상태로 남아 있을 수 있어요.

이 문제를 해결하기 위해서는 SimpleImputer와 같은 대체 메서드를 사용하여 NaN 값을 적절하게 대체하거나, NaN 값을 가진 행 또는 열을 삭제하여야 해요. 또한, 모델 학습 이전에 NaN 값이 있는지 다시 확인하고 처리하는 것이 중요합니다.

In [12]:
# submission.to_csv("DEAGYUL_11_submission.csv", index=False)

print(submission.head(30))

            일시    평균기온
0   2023-01-01  -1.672
1   2023-01-02  -2.004
2   2023-01-03   3.749
3   2023-01-04   6.793
4   2023-01-05  -4.161
5   2023-01-06  -5.258
6   2023-01-07   1.186
7   2023-01-08   1.791
8   2023-01-09   2.243
9   2023-01-10   2.009
10  2023-01-11  -1.961
11  2023-01-12  -3.703
12  2023-01-13  -3.389
13  2023-01-14  -1.746
14  2023-01-15  -5.045
15  2023-01-16  -4.941
16  2023-01-17  -5.453
17  2023-01-18  -2.795
18  2023-01-19   2.095
19  2023-01-20  -0.712
20  2023-01-21  -6.455
21  2023-01-22  -9.380
22  2023-01-23 -13.142
23  2023-01-24 -12.591
24  2023-01-25  -7.332
25  2023-01-26  -7.521
26  2023-01-27 -10.795
27  2023-01-28 -11.248
28  2023-01-29  -7.859
29  2023-01-30  -0.373
